# Tabular Data

This article will get us started reading and working with tabular data.  The goal is to read in some csv data, do a few basic operations (filtering, aggregation, etc.), and create some basic plots.

In [1]:
]box on -style=max

┌→────────────────┐
│Was ON -style=max│
└─────────────────┘

In [2]:
⍝| echo: false
adult ← 'adult_sample/adult.csv'
cricket ← 'Lock5Data3eCSV/CricketChirps.csv'

# Moving Average

We will start with a moving average as the simplest model.  To do this we need to accomplish a few things:

1. Load data from CSV
1. Filter data
1. Calculate Moving Average
1. Plot

## Load Data from CSV

APL has a CSV function to read csv data in that we can use.  I am using simple stock ticker data for a moving average.

In [3]:
eod ← 'eod-quotemedia.csv'
eod_ar ← ⎕CSV eod '' 4
⍴ eod_ar ⍝ Check Size

┌→───────┐
│490738 3│
└~───────┘

We can get the head of the dataframe by selecting the first 5 rows and 3 columns.  It's handy to have a small sized piece of data as you develop so we store in it's own array also.  `↑` Lets us specify the head slice.

In [4]:
⎕←eod_ar_s ← 5 3 ↑ eod_ar

┌→──────────────────────────────────┐
↓ ┌→───┐ ┌→─────┐ ┌→────────┐ │
│ │date│ │ticker│ │adj_close│ │
│ └────┘ └──────┘ └─────────┘ │
│ ┌→─────────┐ ┌→┐ │
│ │2013-07-01│ │A│ 29.9942 │
│ └──────────┘ └─┘ │
│ ┌→─────────┐ ┌→┐ │
│ │2013-07-02│ │A│ 29.6501 │
│ └──────────┘ └─┘ │
│ ┌→─────────┐ ┌→┐ │
│ │2013-07-03│ │A│ 29.7052 │
│ └──────────┘ └─┘ │
│ ┌→─────────┐ ┌→┐ │
│ │2013-07-05│ │A│ 30.4346 │
│ └──────────┘ └─┘ │
└∊──────────────────────────────────┘

## Filter for 1 ticker

Next, we need to filter for a particular ticker.  The first thing we need to know is where the column we want to sort on is located.  We can of course see that it's the second column, but let's pretend we need to use APL for this to get some  practice.

In [5]:
Let's break that down!

SYNTAX ERROR: Unpaired quote
      Let 's break that down!
          ∧


In [6]:
⎕ ← ticker_loc ← ,/ ⍸ 1⌷eod_ar ∊ ⊂'ticker'

2

Let's break this down a bit:

+ `1⌷eod_ar` Gets the first row - this is our header row
+ `∊ ⊂'ticker'` Determines where 'ticker' is and returns a mask ([0 1 0]
+ `⍸` is applied to get the location of the true values (2)
+ `,/` Applies a concatenation to flatten from an array to a value


The next step is getting a truth mask (ie `[0 1 1 0 0]`) that tells us which rows contain the value to filter for, in this case `AAPL`

In [7]:
tickers ← eod_ar[;ticker_loc] ∊ ⊂ 'AAPL'

Similar to our first filtering section, `∊ ⊂ 'AAPL'` is checking membership of `eod_ar[;ticker_loc]` to return the mask.  

We can see we found 1009 rows out of 490,738 that had this ticker.  


In [8]:
⍝ We have a truth array (0/1) of whether row contains 'AAPL' or not
('size',⍴ tickers) , 'Found',+/tickers 
'min',(⌈/ tickers) , 'max',⌊/tickers

┌→─────────────────────┐
│size 490738 Found 1009│
└+─────────────────────┘

┌→──────────┐
│min 1 max 0│
└+──────────┘

`+/` distributes the `+` sign between each item in the list giving us a sum of the vector.  The same approach can be used for `⌊` and `⌈` to get minimum and maximum values (0 and 1).  `⍴` gives us our shape.

Next we need to use this mask to actually filter out data.  This is quite easy and we can pass our mask with our full array to do that filtering using `⌿`

In [9]:
AAPL ← tickers ⌿ eod_ar

So when we put that together our full solution is:

In [10]:
col_loc ← ,/ ⍸ 1⌷eod_ar ∊ ⊂'ticker'           ⍝ Column Location
AAPL ← eod_ar ⌿⍨ eod_ar[;col_loc] ∊ ⊂ 'AAPL'  ⍝ Filter Array
5 3 ↑ AAPL                                    ⍝ Head of array

┌→────────────────────────────┐
↓ ┌→─────────┐ ┌→───┐ │
│ │2013-07-01│ │AAPL│ 53.1092 │
│ └──────────┘ └────┘ │
│ ┌→─────────┐ ┌→───┐ │
│ │2013-07-02│ │AAPL│ 54.3122 │
│ └──────────┘ └────┘ │
│ ┌→─────────┐ ┌→───┐ │
│ │2013-07-03│ │AAPL│ 54.612 │
│ └──────────┘ └────┘ │
│ ┌→─────────┐ ┌→───┐ │
│ │2013-07-05│ │AAPL│ 54.1734 │
│ └──────────┘ └────┘ │
│ ┌→─────────┐ ┌→───┐ │
│ │2013-07-08│ │AAPL│ 53.8658 │
│ └──────────┘ └────┘ │
└∊────────────────────────────┘

## Calculate Moving Average

We will calculate a 90 period moving average over the stock prices for the AAPL ticker we just filtered for.

In [11]:
ws ← 90

Let's start with calculating a moving sum.  Instead of using `+/` to sum over the full list, we can use `+⌿` to get a moving sum based on the `ws` we give it.

In [12]:
movsum ← ws +⌿ AAPL[;3]

When we have a moving sum, a moving average is easy.  Simply divide the moving sum by the window size to get an average.

In [13]:
movavg ← movsum ÷ ws
'movavg length:',⍴ movavg
'AAPL length:', ⍴ AAPL[;3]

┌→─────────────────┐
│movavg length: 920│
└+─────────────────┘

┌→────────────────┐
│AAPL length: 1009│
└+────────────────┘

Our moving average is shorter than our original data because we are not calculating when we don't have enough data at the beginning of our time period.  Let's pad the beginning with the raw values.

In [14]:
padded ← ((ws-1) ↑ AAPL[;3]) , movavg
'padded length:',⍴ padded
'AAPL length:', ⍴ AAPL[;3]

┌→──────────────────┐
│padded length: 1009│
└+──────────────────┘

┌→────────────────┐
│AAPL length: 1009│
└+────────────────┘

So the full moving average with padding back to original size looks like this

In [15]:
movavg ← (ws ↑ AAPL[;3]) , ws ÷⍨ ws +⌿ AAPL[;3]

## Plot

We can not plot our original data with our moving average.  It's a bit annoying I have to pass in `movavg` twice to get it to work.  It is probably something I am not understanding about how plotting works in APL.

In [16]:
]Plot AAPL[;3] movavg movavg

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 
 for X-axis labels 
 
 0 
 100 
 200 
 300 
 400 
 500 
 600 
 700 
 800 
 900 
 1000 
 1100 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M3794 834L3791 841 3787 797 3784 840 3780 798 3777 788 3774 801 3770 796 3767 814 3764 787 3760 869 3757 828 3754 811 3750 782 3747 806 3743 734 3740 612 3737 604 3733 623 3730 633 3727 603 3723 649 3720 657 3717 639 3713 640 3710 635 3706 645 3703 636 3700 632 3696 651 3693 662 3690 708 3686 602 3683 598 3680 590 3676 633 3673 660 3669 645 3666 665 3663 746 3659 795 3656 785 3653 775 3649 794 3646 853 3643 850 3639 853 3636 836 3632 853 3629 881 3626 878 3622 913 3619 903 3616 890 3612 906 3609 891 3606 894 3602 863 3599 859 3595 853 3592 846 3589 831 3585 852 3582 853 3579 848 3575 844 3572 850 3569 909 3565 914 3562 908 3558 898 3555 930 3552 897 3548 927 3545 913 3542 918 3538 947 3535 943 3532 944 3528 953 3525 947 3521 936 3518 940 3515 931 3511 948 3508 931 3505 987 3501 989 3498 994 3495 997 3491 985 3488 993 3484 1013 3481 1021 3478 1017 3474 1027 3471 1062 3468 1086 3464 1079 3461 1099 3458 1109 3454 1134 3451 1158 3447 1169 3444 1165 3441 1313 3437 1307 3434 1301 3431 1301 3427 1302 3424 1341 3421 1338 3417 1340 3414 1344 3410 1340 3407 1340 3404 1359 3400 1355 3397 1345 3394 1358 3390 1360 3387 1382 3384 1408 3380 1420 3377 1417 3373 1424 3370 1406 3367 1405 3363 1395 3360 1410 3357 1414 3353 1399 3350 1401 3347 1407 3343 1421 3340 1424 3336 1437 3333 1437 3330 1474 3326 1461 3323 1498 3320 1520 3316 1542 3313 1559 3310 1543 3306 1551 3303 1530 3299 1511 3296 1509 3293 1505 3289 1516 3286 1505 3283 1506 3279 1539 3276 1542 3273 1541 3269 1599 3266 1627 3262 1572 3259 1585 3256 1523 3252 1519 3249 1532 3246 1564 3242 1544 3239 1520 3236 1522 3232 1481 3229 1478 3225 1463 3222 1441 3219 1387 3215 1399 3212 1420 3209 1411 3205 1410 3202 1403 3199 1401 3195 1400 3192 1413 3188 1406 3185 1426 3182 1431 3178 1471 3175 1474 3172 1491 3168 1492 3165 1502 3162 1491 3158 1509 3155 1473 3151 1490 3148 1495 3145 1498 3141 1460 3138 1481 3135 1481 3131 1481 3128 1454 3125 1441 3121 1517 3118 1593 3114 1643 3111 1689 3108 1641 3104 1585 3101 1598 3098 1597 3094 1617 3091 1630 3088 1632 3084 1616 3081 1613 3077 1601 3074 1591 3071 1575 3067 1582 3064 1565 3061 1570 3057 1568 3054 1564 3051 1562 3047 1588 3044 1593 3040 1592 3037 1576 3034 1585 3030 1602 3027 1634 3024 1647 3020 1673 3017 1642 3014 1679 3010 1676 3007 1704 3003 1828 3000 1815 2997 1789 2993 1774 2990 1763 2987 1765 2983 1766 2980 1787 2977 1786 2973 1824 2970 1814 2966 1822 2963 1828 2960 1843 2956 1851 2953 1862 2950 1844 2946 1850 2943 1873 2940 1890 2936 1920 2933 1893 2929 1840 2926 1851 2923 1843 2919 1860 2916 1855 2913 1811 2909 1819 2906 1813 2903 1815 2899 1786 2896 1769 2892 1783 2889 1782 2886 1790 2882 1804 2879 1808 2876 1793 2872 1765 2869 1755 2866 1754 2862 1770 2859 1804 2855 1833 2852 1857 2849 1877 2845 1870 2842 1892 2839 1884 2835 1950 2832 1954 2829 1911 2825 1893 2822 1905 2818 1907 2815 1896 2812 1889 2808 1869 2805 1900 2802 1898 2798 1876 2795 1817 2792 1689 2788 1674 2785 1662 2781 1657 2778 1634 2775 1638 2771 1627 2768 1580 2765 1536 2761 1537 2758 1568 2755 1596 2751 1604 2748 1606 2744 1558 2741 1581 2738 1555 2734 1577 2731 1597 2728 1586 2724 1623 2721 1672 2718 1663 2714 1653 2711 1642 2707 1658 2704 1658 2701 1660 2697 1657 2694 1684 2691 1725 2687 1730 2684 1751 2681 1752 2677 1754 2674 1737 2670 1715 2667 1745 2664 1760 2660 1764 2657 1840 2654 1835 2650 1838 2647 1851 2644 1879 2640 1836 2637 1853 2633 1848 2630 1811 2627 1841 2623 1893 2620 1899 2617 1887 2613 1873 2610 1873 2607 1892 2603 1841 2600 1857 2596 1893 2593 1855 

┌⊖┐
⌽0│
└~┘

## Full Solution

In [17]:
col_loc ← ,/ ⍸ 1⌷eod_ar ∊ ⊂'ticker'       
AAPL ← eod_ar ⌿⍨ eod_ar[;col_loc] ∊ ⊂ 'AAPL'  
movavg ← (ws ↑ AAPL[;3]) , ws ÷⍨ ws +⌿ AAPL[;3]
eod_ar ← ⎕CSV eod '' 4
]Plot AAPL[;3] movavg movavg

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 
 for X-axis labels 
 
 0 
 100 
 200 
 300 
 400 
 500 
 600 
 700 
 800 
 900 
 1000 
 1100 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M3794 834L3791 841 3787 797 3784 840 3780 798 3777 788 3774 801 3770 796 3767 814 3764 787 3760 869 3757 828 3754 811 3750 782 3747 806 3743 734 3740 612 3737 604 3733 623 3730 633 3727 603 3723 649 3720 657 3717 639 3713 640 3710 635 3706 645 3703 636 3700 632 3696 651 3693 662 3690 708 3686 602 3683 598 3680 590 3676 633 3673 660 3669 645 3666 665 3663 746 3659 795 3656 785 3653 775 3649 794 3646 853 3643 850 3639 853 3636 836 3632 853 3629 881 3626 878 3622 913 3619 903 3616 890 3612 906 3609 891 3606 894 3602 863 3599 859 3595 853 3592 846 3589 831 3585 852 3582 853 3579 848 3575 844 3572 850 3569 909 3565 914 3562 908 3558 898 3555 930 3552 897 3548 927 3545 913 3542 918 3538 947 3535 943 3532 944 3528 953 3525 947 3521 936 3518 940 3515 931 3511 948 3508 931 3505 987 3501 989 3498 994 3495 997 3491 985 3488 993 3484 1013 3481 1021 3478 1017 3474 1027 3471 1062 3468 1086 3464 1079 3461 1099 3458 1109 3454 1134 3451 1158 3447 1169 3444 1165 3441 1313 3437 1307 3434 1301 3431 1301 3427 1302 3424 1341 3421 1338 3417 1340 3414 1344 3410 1340 3407 1340 3404 1359 3400 1355 3397 1345 3394 1358 3390 1360 3387 1382 3384 1408 3380 1420 3377 1417 3373 1424 3370 1406 3367 1405 3363 1395 3360 1410 3357 1414 3353 1399 3350 1401 3347 1407 3343 1421 3340 1424 3336 1437 3333 1437 3330 1474 3326 1461 3323 1498 3320 1520 3316 1542 3313 1559 3310 1543 3306 1551 3303 1530 3299 1511 3296 1509 3293 1505 3289 1516 3286 1505 3283 1506 3279 1539 3276 1542 3273 1541 3269 1599 3266 1627 3262 1572 3259 1585 3256 1523 3252 1519 3249 1532 3246 1564 3242 1544 3239 1520 3236 1522 3232 1481 3229 1478 3225 1463 3222 1441 3219 1387 3215 1399 3212 1420 3209 1411 3205 1410 3202 1403 3199 1401 3195 1400 3192 1413 3188 1406 3185 1426 3182 1431 3178 1471 3175 1474 3172 1491 3168 1492 3165 1502 3162 1491 3158 1509 3155 1473 3151 1490 3148 1495 3145 1498 3141 1460 3138 1481 3135 1481 3131 1481 3128 1454 3125 1441 3121 1517 3118 1593 3114 1643 3111 1689 3108 1641 3104 1585 3101 1598 3098 1597 3094 1617 3091 1630 3088 1632 3084 1616 3081 1613 3077 1601 3074 1591 3071 1575 3067 1582 3064 1565 3061 1570 3057 1568 3054 1564 3051 1562 3047 1588 3044 1593 3040 1592 3037 1576 3034 1585 3030 1602 3027 1634 3024 1647 3020 1673 3017 1642 3014 1679 3010 1676 3007 1704 3003 1828 3000 1815 2997 1789 2993 1774 2990 1763 2987 1765 2983 1766 2980 1787 2977 1786 2973 1824 2970 1814 2966 1822 2963 1828 2960 1843 2956 1851 2953 1862 2950 1844 2946 1850 2943 1873 2940 1890 2936 1920 2933 1893 2929 1840 2926 1851 2923 1843 2919 1860 2916 1855 2913 1811 2909 1819 2906 1813 2903 1815 2899 1786 2896 1769 2892 1783 2889 1782 2886 1790 2882 1804 2879 1808 2876 1793 2872 1765 2869 1755 2866 1754 2862 1770 2859 1804 2855 1833 2852 1857 2849 1877 2845 1870 2842 1892 2839 1884 2835 1950 2832 1954 2829 1911 2825 1893 2822 1905 2818 1907 2815 1896 2812 1889 2808 1869 2805 1900 2802 1898 2798 1876 2795 1817 2792 1689 2788 1674 2785 1662 2781 1657 2778 1634 2775 1638 2771 1627 2768 1580 2765 1536 2761 1537 2758 1568 2755 1596 2751 1604 2748 1606 2744 1558 2741 1581 2738 1555 2734 1577 2731 1597 2728 1586 2724 1623 2721 1672 2718 1663 2714 1653 2711 1642 2707 1658 2704 1658 2701 1660 2697 1657 2694 1684 2691 1725 2687 1730 2684 1751 2681 1752 2677 1754 2674 1737 2670 1715 2667 1745 2664 1760 2660 1764 2657 1840 2654 1835 2650 1838 2647 1851 2644 1879 2640 1836 2637 1853 2633 1848 2630 1811 2627 1841 2623 1893 2620 1899 2617 1887 2613 1873 2610 1873 2607 1892 2603 1841 2600 1857 2596 1893 2593 1855 

┌⊖┐
⌽0│
└~┘